## Machine Learning 프로젝트 수행을 위한 코드 구조화

`(분류, 회귀 Task)`

- ML project를 위해서 사용하는 템플릿 코드를 만듭니다.

1. **필요한 라이브러리와 데이터를 불러옵니다.**


2. **EDA를 수행합니다.** 이 때 EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. **전처리를 수행합니다.** 이 때 중요한건 **feature engineering**을 어떻게 하느냐 입니다.


4. **데이터 분할을 합니다.** 이 때 train data와 test data 간의 분포 차이가 없는지 확인합니다.


5. **학습을 진행합니다.** 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. **hyper-parameter tuning을 수행합니다.** 원하는 목표 성능이 나올 때 까지 진행합니다. 검증 단계를 통해 지속적으로 **overfitting이 되지 않게 주의**하세요.


7. **최종 테스트를 진행합니다.** 데이터 분석 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.

## 1. 라이브러리, 데이터 불러오기

In [140]:
                            # 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMClassifier
from lightgbm.sklearn import LGBMRegressor

# 상관관계 분석, VIF : 다중공선성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

In [141]:
# flag setting
data_reducing = False ## memory reducing technique
feature_reducing = False ## feature extraction (curse of dimensionality)

In [142]:
# 데이터를 불러옵니다.
train = pd.read_csv('../input/dacon-elec-usage-prediction/train.csv', encoding='cp949',
                   parse_dates=["date_time"])
test = pd.read_csv('../input/dacon-elec-usage-prediction/test.csv', encoding='cp949',
                  parse_dates=["date_time"])

In [143]:
train.columns

In [144]:
train.columns = ['num','date_time', 'target', 'temperature', 'windspeed',
                'humidity', 'precipitation', 'insolation',
                'cool_flag', 'solar_flag']
test.columns = ['num','date_time', 'temperature', 'windspeed',
                'humidity', 'precipitation', 'insolation',
                'cool_flag', 'solar_flag']

## 2. EDA

- 데이터에서 찾아야 하는 기초적인 내용들을 확인합니다.


- class imbalance, target distribution, outlier, correlation을 확인합니다.

In [145]:
## On your Own
## 1. 데이터 크기 확인


## 2. 결측치 확인


## 3. dtype이 object인 column 확인



## 4. target distribution
## Q1. target value가 튀는 데이터는 없나요?
## Q2. target value의 평균은 얼마인가요?
## Q3. 122400개의 데이터에 대해서 target value의 평균을 보는게 의미가 있을까요?
## Q4. 건물별 전력사용량이 가장 높은 건물은 무엇인가요?
## Q5. 요일 별 전력사용량의 패턴이 비슷한가요?
...
...

In [146]:
plt.figure(figsize=(16, 10))
sns.lineplot(data=train, x="date_time", y="target", ci=None)
plt.show()

In [147]:
#for n in train.num.unique():
#    building = train[train.num == n]
#    plt.figure(figsize=(16, 6))
#    plt.title(f"\nEnergy Usage Pattern in Building No.{n}", fontsize=12)
#    sns.lineplot(data=building, x="date_time", y="target")
#    plt.show()

이런 식으로 여러가지 그래프를 그려가며, 데이터에 대한 인사이트를 얻습니다!

### 3. 전처리

#### 결측치 처리


1) 건물 정보
- 비전기냉방설비운영 / 태양광보유


- train data의 건물 정보가 바뀌는가?  ----->  test data의 건물 정보가 바뀌었는가?


> 만약에, train, test 데이터가 모두 건물 정보에서 변함이 없다면, 모두 같은 값으로 채워준다.

> 만약에, train, test 데이터가 건물 정보에서 변화가 있다면, 언제 바뀌었는지 체크를 한다.

In [148]:
# 건물 정보 확인
# 1. train data의 건물 정보가 변하는지 확인   (변하지 않음을 확인!)
building_info = train[["num", "cool_flag", "solar_flag"]].drop_duplicates()
building_info

In [149]:
# 2. test data에서 건물 정보가 변하는지 체크
building_test = test[["num", "cool_flag", "solar_flag"]]
test_cool = test.loc[~test.cool_flag.isnull(), ["num", "cool_flag"]]
test_solar = test.loc[~test.solar_flag.isnull(), ["num", "solar_flag"]]

In [150]:
test_cool = test_cool.drop_duplicates()
test_solar = test_solar.drop_duplicates()
#pd.merge(building_info, test_cool, on="num")  ## INNER JOIN
temp = pd.merge(building_info, test_solar, on="num")
temp2 = pd.merge(building_info, test_cool, on="num")
(temp.solar_flag_x == temp.solar_flag_y).mean()
(temp2.cool_flag_x == temp2.cool_flag_y).mean()

In [151]:
# building_info <---> test
test = test.drop(columns=["cool_flag", "solar_flag"])
test

In [152]:
# INNER JOIN으로 cool_flag와 solar_flag를 채워줍니다.
test = pd.merge(test, building_info, on="num")
test

2) 기상 정보


- 시간에 따른 정보의 흐름이 있는 데이터이기 때문에(시계열 데이터), 쿨하게 85일치의 평균값으로 채워주면 안됩니다.


- 대신에 데이터의 흐름을 보고 맞게 채워주는 방식을 선택하려고 합니다.


- 선형 보간법(linear interpolation)을 사용해서 결측치를 채웁니다.

In [153]:
test.temperature = test.temperature.interpolate(method="linear")
test.windspeed = test.windspeed.interpolate(method="linear")
test.humidity = test.humidity.interpolate(method="linear")
test.precipitation = test.precipitation.interpolate(method="linear")
test.insolation = test.insolation.interpolate(method="linear")
test

In [154]:
test.info()

#### Time feature

- 월, 일, 요일, 시간 정보를 date_time column으로 부터 생성합니다.

In [155]:
train["month"] = train.date_time.dt.month
train["day"] = train.date_time.dt.day
train["hour"] = train.date_time.dt.hour
train["dow"] = train.date_time.dt.dayofweek

test["month"] = test.date_time.dt.month
test["day"] = test.date_time.dt.day
test["hour"] = test.date_time.dt.hour
test["dow"] = test.date_time.dt.dayofweek

#### feature extraction

- 차원의 저주를 해결하거나, 데이터의 feature 조합을 이용하는 새로운 feature를 생성할 때, PCA를 사용합니다.

- 분석에 사용할 feature를 선택하는 과정도 포함합니다.

In [156]:
# PCA 적용
from sklearn.decomposition import PCA

if feature_reducing:
    pca = PCA(n_components=0.9) # PCA(n_components=6)
    pca_data = pca.fit_transform(X)

### 4. 학습 데이터 분할

과제는 해당 코드를 기반으로, 서로 나뉘어져있는 데이터를 따로 학습을 하여 2개의 학습 모델을 만들고, 두 개의 예측 결과를 합쳐서 submission.csv를 만드는 것입니다.
3, 9, 15번 건물 데이터는 10,000개보다 적으므로 RandomForest를 이용하여 학습하고, 나머지 데이터는 LGBM으로 학습을 진행합니다.
예측을 수행할 때는 건물별로 나뉘는 ID를 토대로 다른 모델을 사용하여 예측 결과를 만들어야 합니다.

In [157]:
# 첫번째 테스트용으로 사용하고, 실제 학습시에는 K-Fold CV를 사용합니다.
# train : test = 8 : 2
from sklearn.model_selection import train_test_split

# 건물 번호가 3, 9, 15인 train data
train_3915 = train[train.num.isin([3, 9, 15])]
# 나머지
train_ = train[~train.num.isin([3, 9, 15])]

X_3915 = train_3915.drop(columns=["num", "date_time", "target"])
y_3915 = train_3915.target

X = train_.drop(columns=["num", "date_time", "target"])
y = train_.target

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_3915.shape, X.shape, y_3915.shape, y.shape)

In [158]:
# 테스트용 데이터 나누기

# 건물 번호가 3, 9, 15인 train data
test_3915 = test[test.num.isin([3, 9, 15])]
# 나머지
test_ = test[~test.num.isin([3, 9, 15])]

X_test_3915 = test_3915.drop(columns=["num", "date_time"])

X_test = test_.drop(columns=["num", "date_time"])

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_test_3915.shape, X_test.shape)

### 5. 학습 및 평가

In [171]:
# metric은 그때마다 맞게 바꿔줘야 합니다.
def smape(target, preds):
    '''
    Function to calculate SMAPE
    '''
    n = len(preds)
    masked_arr = ~((preds==0)&(target==0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds-target)
    denom = np.abs(preds)+np.abs(target)
    smape_val = (200*np.sum(num/denom))/n
    return smape_val

evaluation_metric = smape
#evaluation_metric = mean_absolute_percentage_error

In [132]:
# 간단하게 LightGBM 테스트
# 적당한 hyper-parameter 조합을 두었습니다. (항상 best는 아닙니다. 예시입니다.)

param_grid = {
    "max_bin" : 20,
    "learning_rate" : 0.0025,
    "objective" : "regression",
    "boosting_type" : "gbdt",
    "metric" : "mae",
    "sub_feature" : 0.345,
    "bagging_fraction" : 0.85,
    "bagging_freq" : 40,
    "num_leaves" : 512,
    "min_data" : 500,
    "min_hessian" : 0.05,
    "verbose" : 2,
    "feature_fraction_seed" : 2,
    "bagging_seed" : 3
}

model = LGBMRegressor(**param_grid)

In [134]:
print("\nFitting LightGBM...")
model.fit(X, y)

In [165]:
gcv.best_params_

In [164]:
# metric은 그때마다 맞게 바꿔줘야 합니다.
evaluation_metric = 

In [166]:
print("Prediction with Best Estimator")
gcv_pred_train = gcv.predict(X_3915)
#pred_test = model.predict(x_test)


gcv_train_score = evaluation_metric(y_3915, pred_train)
#test_score = evaluation_metric(y_test, pred_test)

print("Train Score : %.4f" % train_score)
#print("Test Score : %.4f" % test_score)

In [163]:
print("Prediction")
pred_train = model.predict(X_3915)
#pred_test = model.predict(x_test)


train_score = evaluation_metric(y_3915, pred_train)
#test_score = evaluation_metric(y_test, pred_test)

print("Train Score : %.4f" % train_score)
print("Test Score : %.4f" % test_score)

### 6. Hyper-parameter Tuning

> GridSearchCV

** LightGBM의 hyperparameter **

[Official Documentation] https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html 

[Blog 1] https://smecsm.tistory.com/133

[Blog 2] https://towardsdatascience.com/kagglers-guide-to-lightgbm-hyperparameter-tuning-with-optuna-in-2021-ed048d9838b5

[Blog 3] https://nurilee.com/2020/04/03/lightgbm-definition-parameter-tuning/

In [168]:
# GridSearchCV를 이용하여 가장 좋은 성능을 가지는 모델을 찾아봅시다. (이것은 첫번째엔 선택입니다.)
# Lightgbm은 hyper-parameter의 영향을 많이 받기 때문에, 저는 보통 맨처음에 한번 정도는 가볍게 GCV를 해봅니다.
# 성능 향상이 별로 없다면, lightgbm으로 돌린 대략적인 성능이 이 정도라고 생각하면 됩니다.
# 만약 성능 향상이 크다면, 지금 데이터는 hyper-parameter tuning을 빡빡하게 하면 성능 향상이 많이 이끌어 낼 수 있습니다.

from sklearn.model_selection import GridSearchCV

param_grid = {
    "max_depth" : [16, None],
    "n_estimators" : [100, 300],
    "max_bin" : [20],
    "learning_rate" : [0.001, 0.003],
    "objective" : ["regression"],
    "boosting_type" : ["gbdt"],
    "metric" : ["mae"],
    "sub_feature" : [0.345],
    "bagging_fraction" : [0.7, 0.75, 0.85],
    "bagging_freq" : [40],
    "num_leaves" : [256, 512],
    "min_data" : [500],
    "verbose" : [-1], # 필수
    "min_hessian" : [0.05],
    "feature_fraction_seed" : [2],
    "bagging_seed" : [3]
}


gcv = GridSearchCV(estimator=model, param_grid=param_grid, cv=5,
                  n_jobs=-1, verbose=1)

gcv.fit(X_3915, y_3915)
print("Best Estimator : ", gcv.best_estimator_)

In [173]:
print("Prediction with Best Estimator")
gcv_pred_train = gcv.predict(X_3915)
#gcv_pred_test = gcv.predict(x_test)

gcv_train_score = evaluation_metric(y_3915, gcv_pred_train)
#gcv_test_score = evaluation_metric(y_test, gcv_pred_test)

print("Train MAE Score : %.4f" % gcv_train_score)
#print("Test MAE Score : %.4f" % gcv_test_score)

In [174]:
print("Performance Gain") # 이걸로 성능 향상 확인.
print("in train : ", (train_score - gcv_train_score))
#print("in test : ", (test_score - gcv_test_score))

> optuna를 사용해봅시다 !

In [ ]:
def optimizer(trial, X, y, K):
    # 조절할 hyper-parameter 조합을 적어줍니다.
    n_estimators = 
    max_depth = 
    max_features = 
    
    
    # 원하는 모델을 지정합니다, optuna는 시간이 오래걸리기 때문에 저는 보통 RF로 일단 테스트를 해본 뒤에 LGBM을 사용합니다.
    model = RandomForestRegressor(n_estimators=n_estimators,
                                 max_depth=max_depth,
                                 max_features=max_features)
    
    
    # K-Fold Cross validation을 구현합니다.
    folds = KFold(n_splits=K)
    losses = []
    
    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        loss = mean_absolute_error(y_val, preds)
        losses.append(loss)
    
    
    # K-Fold의 평균 loss값을 돌려줍니다.
    return np.mean(losses)

In [ ]:
K = # Kfold 수
opt_func = partial(optimizer, X=X_train, y=y_train, K)

study = optuna.create_study(direction="minimize") # 최소/최대 어느 방향의 최적값을 구할 건지.
study.optimize(opt_func, n_trials=5)

In [ ]:
# optuna가 시도했던 모든 실험 관련 데이터
study.trials_dataframe()

In [ ]:
print("Best Score: %.4f" % study.best_value) # best score 출력
print("Best params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들

In [ ]:
# 실험 기록 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

### 7. 테스트 및 제출 파일 생성

In [ ]:
model = RandomForestRegressor(n_estimators=study.best_trial.params["n_estimators"],
                                 max_depth=study.best_trial.params["max_depth"],
                                 max_features=study.best_trial.params["max_features"])

model.fit(X_train, y_train)
preds = model.predict(X_test)
preds

In [ ]:
X_test # 원본 데이터랑 id가 맞는지 확인 해보기!

In [ ]:
submission = pd.DataFrame() # submission을 생성합니다.
submission

In [ ]:
submission.reset_index(drop=True).to_csv("submission.csv", index=False)